 # Multi-Step Agentic RAG with Azure AI Services
 
 This notebook demonstrates a Multi-Step Agentic RAG (Retrieval Augmented Generation) system that provides a cardiology-focused AI assistant with access to multiple data sources:

 1. **Structured Data** (Azure SQL Database) - Patient medical records and information
 2. **Unstructured Data** (Azure AI Search) - Medical guidelines from the American College of Cardiology
 3. **Web Data** (Azure AI Agent Service with Bing Grounding Tool) - Real-time web information
 
 The system uses Azure OpenAI's function/tool calling capabilities to orchestrate the retrieval process and generate comprehensive responses to medical queries.

In [ ]:
!pip install -r requirements.txt

## 1. Import Libraries and Set Up Environment
 We'll start by importing the necessary libraries and loading environment variables.


In [32]:
import os
import json
import pandas as pd
import pyodbc
import sqlalchemy
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import BingGroundingTool
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from dotenv import load_dotenv
from openai import AzureOpenAI
from rich.console import Console
from rich.panel import Panel

load_dotenv()
console = Console()

## 2. Configure Azure Services

Next, we set up all the necessary configurations for the Azure services we'll be using.


In [33]:
# Azure OpenAI configuration
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY", "your-azure-openai-api-key")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-10-21")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT", "https://your-azure-openai-endpoint.openai.azure.com/")
AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME", "gpt-4o")

# Azure AI Search configuration
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT", "https://your-search-service.search.windows.net")
AZURE_SEARCH_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY", "your-azure-search-key")
SEARCH_INDEX_NAME = "acc-guidelines-index"

# Azure AI Project configuration for Bing Grounding
AZURE_CONNECTION_STRING = os.getenv("AZURE_CONNECTION_STRING", "your-azure-connection-string")
BING_CONNECTION_NAME = os.getenv("BING_CONNECTION_NAME", "fsunavalabinggrounding")

# Azure SQL connection details (for patient data)
server = os.getenv("AZURE_SQL_SERVER_NAME")
database = os.getenv("AZURE_SQL_DATABASE_NAME")
username = os.getenv("AZURE_SQL_USER_NAME")
password = os.getenv("AZURE_SQL_PASSWORD")
driver = '{ODBC Driver 17 for SQL Server}'

# Create an Azure SQL connection string
AZURE_SQL_CONNECTION_STRING = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}"

# Initialize the Azure OpenAI client
openai_client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
)

## 3. Define Tool Functions

Now we'll implement the three tool functions that our agent will use to access different data sources.

### 3.1 Unstructured Data Tool: ACC Guidelines Search via Azure AI Search

This tool retrieves relevant American College of Cardiology guidelines using Azure AI Search with semantic and vector search capabilities.


In [34]:
def search_acc_guidelines(query: str) -> str:
    """
    Searches the Azure AI Search index 'acc-guidelines-index' 
    for relevant American College of Cardiology (ACC) guidelines.
    """
    credential = AzureKeyCredential(AZURE_SEARCH_KEY)
    client = SearchClient(
        endpoint=AZURE_SEARCH_ENDPOINT,
        index_name=SEARCH_INDEX_NAME,
        credential=credential,
    )
    
    results = client.search(
        search_text=query,
        vector_queries=[
            VectorizableTextQuery(
                text=query,
                k_nearest_neighbors=10,   # Adjust as needed
                fields="embedding"   # Adjust based on your index schema
            )
        ],
        query_type="semantic",
        semantic_configuration_name="default",
        search_fields=["chunk"],
        top=10,
        include_total_count=True
    )
    
    retrieved_texts = []
    for result in results:
        content_chunk = result.get("chunk", "")
        retrieved_texts.append(content_chunk)
    
    context_str = "\n".join(retrieved_texts) if retrieved_texts else "No relevant guidelines found."
    
    console.print(
        Panel(
            f"Tool Invoked: ACC Guidelines Search\nQuery: {query}",
            style="bold yellow"
        )
    )
    return context_str

 ### 3.2 Web Data Tool: Bing Web Grounding via Azure AI Agent Service
 
 This tool uses the Azure AI Agent Service with the Bing Web Grounding Tool to search for real-time information from the web. This is a significant improvement over direct Bing Search API calls, as it leverages Azure's AI Agent framework for better context and grounding and remains compliant with Bing's Terms of Use.

In [44]:
def search_bing_grounding(query: str) -> str:
    """
    Searches the public web using the Bing Web Grounding Tool via Azure AI Agent Service.
    Returns information about recent updates from the web.
    """
    # Create an Azure AI Client
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=AZURE_CONNECTION_STRING,
    )
    
    try:
        with project_client:
            # Get the Bing connection
            bing_connection = project_client.connections.get(
                connection_name=BING_CONNECTION_NAME
            )
            conn_id = bing_connection.id
            
            # Initialize agent bing tool
            bing = BingGroundingTool(connection_id=conn_id)
            
            # Create agent with the bing tool
            agent = project_client.agents.create_agent(
                model="gpt-4o", # gpt-4o-mini not supported at this time so we'll use GPT-4o
                name="bing-search-agent",
                instructions=f"Search the web for information about: {query}. Provide a concise but comprehensive summary.",
                tools=bing.definitions,
                headers={"x-ms-enable-preview": "true"}
            )
            
            # Create thread for communication
            thread = project_client.agents.create_thread()
            
            # Create message to thread
            project_client.agents.create_message(
                thread_id=thread.id,
                role="user",
                content=query,
            )
            
            # Create and process agent run
            run = project_client.agents.create_and_process_run(
                thread_id=thread.id, 
                assistant_id=agent.id
            )
            
            if run.status == "failed":
                result_text = f"Bing search failed: {run.last_error}"
            else:
                # Fetch messages to get the response
                messages = project_client.agents.list_messages(thread_id=thread.id)
                # Get the last assistant message
                assistant_messages = [m for m in messages.get('data', []) if m.get('role') == 'assistant']
                if assistant_messages:
                    # Extract the text content from the last assistant message
                    content_list = assistant_messages[-1].get('content', [])
                    result_text = ""
                    for content_item in content_list:
                        if isinstance(content_item, dict) and 'text' in content_item:
                            result_text += content_item.get('text', "")
                    
                    if not result_text:
                        result_text = "No results found."
                else:
                    result_text = "No results found."
            
            # Clean up resources
            project_client.agents.delete_agent(agent.id)
            
    except Exception as e:
        result_text = f"Bing search failed with error: {str(e)}"
    
    console.print(
        Panel(
            f"Tool Invoked: Bing Grounding Search\nQuery: {query}",
            style="bold magenta"
        )
    )
    return result_text

### 3.3 Structured Data Tool: Patient Data Query via Azure SQL

This tool queries a SQL database containing patient medical data, transforming natural language questions into SQL queries.


In [45]:
def lookup_patient_data(query: str) -> str:
    """
    Queries the 'PatientMedicalData' table in Azure SQL and returns the results as a string.
    'query' should be a valid SQL statement.
    This version uses SQLAlchemy to create an engine, which is fully supported by pandas.read_sql.
    """
    try:
        # Construct the connection URI for SQLAlchemy
        connection_uri = (
            f"mssql+pyodbc://{username}:{password}@{server}/{database}"
            "?driver=ODBC+Driver+17+for+SQL+Server"
        )
        engine = sqlalchemy.create_engine(connection_uri)
        
        # Use the engine in pandas.read_sql, which avoids the warning
        df = pd.read_sql(query, engine)
        if df.empty:
            return "No rows found."
        return df.to_string(index=False)
    except Exception as e:
        return f"Database error: {str(e)}"

 ## 4. Define Tools for OpenAI Function Calling
 Now we'll configure these tools to work with OpenAI's function calling feature, which allows the LLM to dynamically select and use these tools during conversation.


In [46]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_acc_guidelines",
            "description": "Query the ACC guidelines for official cardiology recommendations. Use keywords related to cardiology conditions, treatments, or guidelines.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Keywords or specific question related to cardiology guidelines."
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search_bing_grounding",
            "description": "Perform a public web search for real-time or external information using Bing Grounding. For example, 'FDA new hyperlipidemia drugs', 'recent hypertension medication approvals'.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "General query to retrieve public data."
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "lookup_patient_data",
            "description": (
                "Query Azure SQL PatientMedicalData table. Schema: "
                "PatientID: INT (PK, Identity), FirstName: VARCHAR(100), LastName: VARCHAR(100), "
                "DateOfBirth: DATE, Gender: VARCHAR(20), ContactNumber: VARCHAR(100), EmailAddress: VARCHAR(100), "
                "Address: VARCHAR(255), City: VARCHAR(100), PostalCode: VARCHAR(20), Country: VARCHAR(100), "
                "MedicalCondition: VARCHAR(255), Medications: VARCHAR(255), Allergies: VARCHAR(255), BloodType: VARCHAR(10), "
                "LastVisitDate: DATE, SmokingStatus: VARCHAR(50), AlcoholConsumption: VARCHAR(50), ExerciseFrequency: VARCHAR(50), "
                "Occupation: VARCHAR(100), Height_cm: DECIMAL(5,2), Weight_kg: DECIMAL(5,2), BloodPressure: VARCHAR(20), "
                "HeartRate_bpm: INT, Temperature_C: DECIMAL(3,1), Notes: VARCHAR(MAX). Use SQL to retrieve patient data."
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Valid SQL query for PatientMedicalData table to get patient information."
                    }
                },
                "required": ["query"]
            }
        }
    }
]

# Map the tool names to the actual function implementations
tool_implementations = {
    "search_acc_guidelines": search_acc_guidelines,
    "search_bing_grounding": search_bing_grounding,
    "lookup_patient_data": lookup_patient_data,
}

## 5. Define System Prompt for the Agent

The system prompt sets the context and instructions for the Agent, defining its role and capabilities.


In [52]:
# ----------------------------
# System Prompt for the Agent
# ----------------------------
SYSTEM_PROMPT = (
    "You are a cardiology-focused AI assistant with access to three tools:\n"
    "1) 'lookup_patient_data' for querying patient records from Azure SQL.\n"
    "2) 'search_acc_guidelines' for official ACC guidelines.\n"
    "3) 'search_bing_grounding' for real-time public information.\n\n"
    "You can call these tools in any order, multiple times if needed, to gather all the context.\n"
    "Stop calling tools only when you have enough information to provide a final, cohesive answer.\n"
    "Then output your final answer to the user."
)


## 6. Implement the Multi-Step Agent

 Now we'll create the agent that orchestrates the entire process. This agent will:

1.  Receive a user query
2.  Decide which tools to call and in what order
3.  Parse the results from each tool
4.  Synthesize a comprehensive answer

The agent can make multiple calls to different tools before providing a final answer, allowing for complex, multi-step reasoning.


In [53]:
def run_multi_step_agent(user_query: str, max_steps: int = 10):
    """
    A multi-step agent with enhanced debugging logs using the new tools syntax.
    """
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_query}
    ]

    for step in range(max_steps):
        console.print(Panel(f"**Step {step+1}**: Starting step {step+1}", title="Step Start", style="bold cyan"))
        
        response = openai_client.chat.completions.create(
            model=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
            messages=messages,
            tools=tools,
            tool_choice="auto",
            max_tokens=8000,
        )
        response_message = response.choices[0].message
        
        console.print(Panel(f"[DEBUG] Step {step+1}: Raw response_message: {response_message}", title="Raw Response", style="dim"))
        
        # IMPORTANT: Append the assistant's message (which includes tool_calls) to the conversation.
        messages.append(response_message)
        
        if response_message.tool_calls:
            for tool_call in response_message.tool_calls:
                function_name = tool_call.function.name
                arguments_str = tool_call.function.arguments
                
                console.print(Panel(f"[DEBUG] Step {step+1}: Tool called: {function_name}", title="Tool Call Info", style="bold magenta"))
                console.print(Panel(f"[DEBUG] Step {step+1}: arguments_str: {arguments_str}", title="Arguments String", style="magenta"))
                
                if arguments_str.strip() == "":
                    function_args = {}
                else:
                    try:
                        function_args = json.loads(arguments_str)
                        console.print(Panel(f"[DEBUG] Step {step+1}: function_args (JSON parsed): {function_args}", title="Parsed Arguments", style="green"))
                    except json.JSONDecodeError as e:
                        function_args = {}
                        console.print(
                            Panel(
                                "Warning: Could not decode tool call arguments; defaulting to empty dict.",
                                title="JSONDecodeError",
                                style="bold red"
                            )
                        )
                        console.print(Panel(f"[DEBUG] Step {step+1}: JSONDecodeError details: {e}", title="JSONDecodeError Detail", style="bold red"))
                        console.print(Panel(f"[DEBUG] Step {step+1}: function_args (defaulted): {function_args}", title="Defaulted Arguments", style="red"))
                
                # Ensure the 'query' key is present
                if "query" not in function_args or not function_args["query"]:
                    function_args["query"] = user_query
                    console.print(Panel(f"[DEBUG] Step {step+1}: 'query' key defaulted to user_query: {user_query}", title="Defaulted Query Key", style="yellow"))
                
                console.print(
                    Panel(
                        f"**Step {step+1}**: LLM calls tool [bold]{function_name}[/bold]\n\n"
                        f"**Arguments**:\n{json.dumps(function_args, indent=2)}",
                        title="Tool Call",
                        style="bold blue"
                    )
                )
                console.print(Panel(f"[DEBUG] Step {step+1}: Calling tool implementation: {function_name} with args: {function_args}", title="Tool Implementation Call", style="bold magenta"))
                
                tool_fn = tool_implementations.get(function_name)
                if tool_fn is None:
                    tool_output = f"[Error] No implementation for tool '{function_name}'."
                else:
                    tool_output = tool_fn(**function_args)
                console.print(Panel(f"[DEBUG] Step {step+1}: Tool output: {tool_output}", title="Tool Output", style="italic blue"))
                
                # Format the tool response message correctly
                messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": str(tool_output),  # Simple string for tool response
                })
        else:
            final_answer = response_message.content
            console.print(
                Panel(
                    final_answer,
                    title="Final Answer",
                    style="bold green",
                    border_style="yellow"
                )
            )
            return

    console.print(
        Panel(
            "Max steps reached without a final answer. Stopping.",
            title="Warning",
            style="bold red"
        )
    )
    return

 ## 7. Execute the Agent with a Sample Query
 
 Now let's run the agent with a cardiology-related question to see how it works. The agent will decide which tools to use and in what order to provide the most comprehensive answer.

In [ ]:
user_question_1 = "What does the ACC recommend as first-line therapy for hypertension in elderly patients?"
run_multi_step_agent(user_question_1)


In [ ]:
user_question_2 = "Any latest FDA approvals or clinical trial facts related to Ozempic and how it impacts cardiovascular diseases?"
run_multi_step_agent(user_question_2)


In [ ]:
user_question_3 = "How many patients have Hypertension and are prescribed Lisinopril?"
# The agent should generate a valid SQL query, for example:
## Note, the answer will depend on the actual data in the database (it should return 1071!)
run_multi_step_agent(user_question_3)


In [54]:
run_multi_step_agent("I have a 79-year-old patient named Gloria Paul with hyperlipidemia. She's on Atorvastatin. Can you confirm her medical details from the database, check the ACC guidelines for hyperlipidemia, and see if there are any new medication updates from the FDA as of Feb 2025? Then give me a summary.")

╭────────────────────────────────────────────────── Step Start ───────────────────────────────────────────────────╮
│ **Step 1**: Starting step 1                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Raw Response ──────────────────────────────────────────────────╮
│ [DEBUG] Step 1: Raw response_message: ChatCompletionMessage(content=None, refusal=None, role='assistant',       │
│ audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_mGaA5JhApyGSEsbOO6VZfPxo',   │
│ function=Function(arguments='{"query": "SELECT * FROM PatientMedicalData WHERE FirstName = \'Gloria\' AND       │
│ LastName = \'Paul\'"}', name='lookup_patient_data'), type='function'),                                          │
│ ChatCompletionMessageToolCall(id='call_m8Tp8mTWUA2OhxGF9UHiwCnT', function=Function(arguments='{"query":        │
│ "hyperlipidemia"}', name='search_acc_guidelines'), type='function'),                                            │
│ ChatCompletionMessageToolCall(id='call_HrMIbfeKN3FT0HloW30qKHNQ', function=Function(arguments='{"query": "FDA   │
│ new hyperlipidemia medications February 2025"}', name='search_bing_grounding'), type='function')])              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tool Call Info ─────────────────────────────────────────────────╮
│ [DEBUG] Step 1: Tool called: lookup_patient_data                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Arguments String ────────────────────────────────────────────────╮
│ [DEBUG] Step 1: arguments_str: {"query": "SELECT * FROM PatientMedicalData WHERE FirstName = 'Gloria' AND       │
│ LastName = 'Paul'"}                                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Parsed Arguments ────────────────────────────────────────────────╮
│ [DEBUG] Step 1: function_args (JSON parsed): {'query': "SELECT * FROM PatientMedicalData WHERE FirstName =      │
│ 'Gloria' AND LastName = 'Paul'"}                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Tool Invoked: Bing Grounding Search                                                                             │
│ Query: FDA new hyperlipidemia medications February 2025                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│ [DEBUG] Step 1: Tool output: No results found.                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Step Start ───────────────────────────────────────────────────╮
│ **Step 2**: Starting step 2                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Raw Response ──────────────────────────────────────────────────╮
│ [DEBUG] Step 2: Raw response_message: ChatCompletionMessage(content='Here is a summary of the findings          │
│ regarding your patient, Gloria Paul, a 79-year-old with hyperlipidemia:\n\n### Patient Details\n- **Name**:     │
│ Gloria Paul\n- **Date of Birth**: July 15, 1944 (Age: 79)\n- **Contact**: 428-674-1336, keymary@example.com\n-  │
│ **Address**: 62908 Fuentes Cove, Lewisberg, MN, 13491, India\n- **Medical Condition**: Hyperlipidemia\n-        │
│ **Medications**: Atorvastatin\n- **Allergies**: None\n- **Blood Type**: O+\n- **Last Visit Date**: March 26,    │
│ 2024\n- **Smoking Status**: Never Smoker\n- **Alcohol Consumption**: Light drinker (1-2 times a week)\n-        │
│ **Occupation**: Administrator\n- **Height**: 176.2 cm\n- **Weight**: 77.7 kg\n- **Blood Pressure**: 120/79      │
│ mmHg\n- **Heart Rate**: 90 bpm\n- **Temperature**: 37.6 °C\n\n### ACC Guidelines for Hyperlipidemia\n1.         │
│ **Statin Use**: \n   - Statin treatment is recommended for all patients with extracranial carotid or vertebral  │
│ atherosclerosis to reduce LDL cholesterol below 100 mg/dL (Level of Evidence: B).\n   - It is reasonable for    │
│ patients to aim for LDL cholesterol levels near or below 70 mg/dL after an ischemic stroke (Level of Evidence:  │
│ B).\n2. **Alternative Therapies**:\n   - If statins fail to achieve targeted LDL levels, adding other           │
│ medications, such as bile acid sequestrants or niacin, is effective (Level of Evidence: B).\n   - Non-statin    │
│ alternatives are reasonable for patients intolerant to statins.\n\n### FDA Updates\nUnfortunately, the search   │
│ for recent information on new hyperlipidemia medications from the FDA as of February 2025 yielded no results.   │
│ It seems that either no new medications have been approved, or the data is currently unavailable.\n\n###        │
│ Conclusion\nGloria is appropriately managed on Atorvastatin, aligning with ACC guidelines. Monitoring her LDL   │
│ levels is essential, and considering further steps if targets are not reached may be necessary. No new          │
│ medications are specifically noted from the FDA for hyperlipidemia, so current therapy should continue unless   │
│ new evidence arises.\n\nIf you need more specific guidance or updates on additional treatment options for her   │
│ condition, feel free to ask!', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None) │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Final Answer ──────────────────────────────────────────────────╮
│ Here is a summary of the findings regarding your patient, Gloria Paul, a 79-year-old with hyperlipidemia:       │
│                                                                                                                 │
│ ### Patient Details                                                                                             │
│ - **Name**: Gloria Paul                                                                                         │
│ - **Date of Birth**: July 15, 1944 (Age: 79)                                                                    │
│ - **Contact**: 428-674-1336, keymary@example.com                                                                │
│ - **Address**: 62908 Fuentes Cove, Lewisberg, MN, 13491, India                                                  │
│ - **Medical Condition**: Hyperlipidemia                                                                         │
│ - **Medications**: Atorvastatin                                                                                 │
│ - **Allergies**: None                                                                                           │
│ - **Blood Type**: O+                                                                                            │
│ - **Last Visit Date**: March 26, 2024                                                                           │
│ - **Smoking Status**: Never Smoker                                                                              │
│ - **Alcohol Consumption**: Light drinker (1-2 times a week)                                                     │
│ - **Occupation**: Administrator                                                                                 │
│ - **Height**: 176.2 cm                                                                                          │
│ - **Weight**: 77.7 kg                                                                                           │
│ - **Blood Pressure**: 120/79 mmHg                                                                               │
│ - **Heart Rate**: 90 bpm                                                                                        │
│ - **Temperature**: 37.6 °C                                                                                      │
│                                                                                                                 │
│ ### ACC Guidelines for Hyperlipidemia                                                                           │
│ 1. **Statin Use**:                                                                                              │
│    - Statin treatment is recommended for all patients with extracranial carotid or vertebral atherosclerosis to │
│ reduce LDL cholesterol below 100 mg/dL (Level of Evidence: B).                                                  │
│    - It is reasonable for patients to aim for LDL cholesterol levels near or below 70 mg/dL after an ischemic   │
│ stroke (Level of Evidence: B).                                                                                  │
│ 2. **Alternative Therapies**:                                                                                   │
│    - If statins fail to achieve targeted LDL levels, adding other medications, such as bile acid sequestrants   │
│ or niacin, is effective (Level of Evidence: B).                                                                 │
│    - Non-statin alternatives are reasonable for patients intolerant to statins.                                 │
│                                                                                                                 │
│ ### FDA Updates                                                                                                 │
│ Unfortunately, the search for recent information on ne